## Project Goal:
#### This Project is related to identify if a fellow will ultimately be placed at a company and how long until a placement can be expected. 

## The project can be classified into two problems:
1) Problem Statement 1 ===>> To identify if a fellow will ultimately be placed at a company 

2) Problem Statement 2 ===>> To predict how long will it take for the candidate to get placed.

In [ ]:
Steps for Analysis Includes:

 Step 1 : Import the libraries and dataset

 Step 2 : See DataFrame and show basic info and statistical interference about the dataset

 Step 3:  Check out the missing values and handle it

 Step 4 : See the Categorical Values and handle it

 Step 5 : Findout independent and dependent variable for Problem Statements 1 and 2 under consideration

 Step 6 : Splitting the dataset into Training and Test Set

 Step 7 : Model Building

 Step 8 : Model Evaluation
    
 Step 9 : Model Deployment 
    
 Step 10 : Conclusion
    

### Importing Libraries and Data

In [275]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
import numpy as np

In [276]:
dataset=pd.read_excel("Data_Pathrise.xlsx",index_col=0)
dataset.head(2)

,pathrise_status,primary_track,cohort_tag,program_duration_days,placed,employment_status,highest_level_of_education,length_of_job_search,biggest_challenge_in_search,professional_experience,work_authorization_status,number_of_interviews,number_of_applications,gender,race
id,,,,,,,,,,,,,,,
1,Active,SWE,OCT19A,NaN,0,Unemployed,Bachelor's Degree,3-5 months,Hearing back on my applications,3-4 years,Canada Citizen,2.0,900,Male,Non-Hispanic White or Euro-American
2,Active,PSO,JAN20A,NaN,0,Unemployed,"Some College, No Degree",3-5 months,Getting past final round interviews,1-2 years,Citizen,6.0,0,Male,Non-Hispanic White or Euro-American


### Dataset info and statistical interference

In [277]:
dataset.shape

(2544, 15)

In [278]:
dataset.columns

Index(['pathrise_status', 'primary_track', 'cohort_tag',
       'program_duration_days', 'placed', 'employment_status ',
       'highest_level_of_education', 'length_of_job_search',
       'biggest_challenge_in_search', 'professional_experience',
       'work_authorization_status', 'number_of_interviews',
       'number_of_applications', 'gender', 'race'],
      dtype='object')

In [279]:
dataset.columns = dataset.columns.str.rstrip()# To remove white spaces at end of column names

In [280]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2544 entries, 1 to 2544
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   pathrise_status              2544 non-null   object 
 1   primary_track                2544 non-null   object 
 2   cohort_tag                   2536 non-null   object 
 3   program_duration_days        1928 non-null   float64
 4   placed                       2544 non-null   int64  
 5   employment_status            2315 non-null   object 
 6   highest_level_of_education   2486 non-null   object 
 7   length_of_job_search         2470 non-null   object 
 8   biggest_challenge_in_search  2520 non-null   object 
 9   professional_experience      2322 non-null   object 
 10  work_authorization_status    2260 non-null   object 
 11  number_of_interviews         2326 non-null   float64
 12  number_of_applications       2544 non-null   int64  
 13  gender            

### To address Problem Statement 1 and Problem Statement 2 and to keep our model simple, we can drop some unnecessary features 

In [282]:
dataset=dataset.drop(["gender","race","cohort_tag","number_of_applications"],axis=1)

### Dealing with Missing values
For Categorical missing values, we replace each missing value by the most occuring value in their respective columns and for numerical missing values, we replace each missing value by mean value in respective columns.

In [283]:
dataset["biggest_challenge_in_search"]=dataset["biggest_challenge_in_search"].replace(np.nan, dataset["biggest_challenge_in_search"].mode()[0])
dataset["highest_level_of_education"]=dataset["highest_level_of_education"].replace(np.nan, dataset["highest_level_of_education"].mode()[0])
dataset["professional_experience"]=dataset["professional_experience"].replace(np.nan, dataset["professional_experience"].mode()[0])
dataset["employment_status"]=dataset["employment_status"].replace(np.nan, dataset["employment_status"].mode()[0])
dataset["number_of_interviews"]=dataset["number_of_interviews"].replace(np.nan, int(dataset["number_of_interviews"].mean()))
dataset["length_of_job_search"]=dataset["length_of_job_search"].replace(np.nan, dataset["length_of_job_search"].mode()[0])
dataset["work_authorization_status"]=dataset["work_authorization_status"].replace(np.nan, dataset["work_authorization_status"].mode()[0])

In [284]:
dataset.isnull().sum()# Since we need to identify program_duration_days, we won't replace missing values in that column.

pathrise_status                  0
primary_track                    0
program_duration_days          616
placed                           0
employment_status                0
highest_level_of_education       0
length_of_job_search             0
biggest_challenge_in_search      0
professional_experience          0
work_authorization_status        0
number_of_interviews             0
dtype: int64

In [285]:
dataset.head(2)

,pathrise_status,primary_track,program_duration_days,placed,employment_status,highest_level_of_education,length_of_job_search,biggest_challenge_in_search,professional_experience,work_authorization_status,number_of_interviews
id,,,,,,,,,,,
1,Active,SWE,NaN,0,Unemployed,Bachelor's Degree,3-5 months,Hearing back on my applications,3-4 years,Canada Citizen,2.0
2,Active,PSO,NaN,0,Unemployed,"Some College, No Degree",3-5 months,Getting past final round interviews,1-2 years,Citizen,6.0


In [286]:
dataset["number_of_interviews"]=dataset["number_of_interviews"].astype(int)

### Feature Engineering 
(Adding a new column dataset["number_of_interviews-binned"] to dataset)

In [289]:
def func1(x):
    if x < 5:
        return "less than 5"
    elif x < 10:
        return "less than 10"
    elif x < 15:
        return "less than 15"
    else:
        return 'more than 15'

In [290]:
dataset["number_of_interviews-binned"]=dataset["number_of_interviews"].apply(func1)

In [291]:
dataset=dataset.drop(["number_of_interviews"],axis=1)

### Dealing with Categorical Values

In [292]:
from sklearn.preprocessing import LabelEncoder
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()
label_encoder3 = LabelEncoder()
label_encoder4 = LabelEncoder()
label_encoder5 = LabelEncoder()
label_encoder6 = LabelEncoder()
label_encoder7 = LabelEncoder()
label_encoder8= LabelEncoder()
dataset["primary_track"]=label_encoder1.fit_transform(dataset["primary_track"].astype(str))
dataset["biggest_challenge_in_search"]=label_encoder2.fit_transform(dataset["biggest_challenge_in_search"].astype(str))
dataset["highest_level_of_education"]=label_encoder3.fit_transform(dataset["highest_level_of_education"].astype(str))
dataset["professional_experience"]=label_encoder4.fit_transform(dataset["professional_experience"].astype(str))
dataset["employment_status"]=label_encoder5.fit_transform(dataset["employment_status"].astype(str))
dataset["number_of_interviews-binned"]=label_encoder6.fit_transform(dataset["number_of_interviews-binned"].astype(str))
dataset["length_of_job_search"]=label_encoder7.fit_transform(dataset["length_of_job_search"].astype(str))
dataset["work_authorization_status"]=label_encoder8.fit_transform(dataset["work_authorization_status"].astype(str))

In [184]:
dataset.head(2)

,pathrise_status,primary_track,program_duration_days,placed,employment_status,highest_level_of_education,length_of_job_search,biggest_challenge_in_search,professional_experience,work_authorization_status,number_of_interviews-binned
id,,,,,,,,,,,
1,Active,4,NaN,0,4,0,1,5,1,0,2
2,Active,3,NaN,0,4,5,1,2,0,1,0


# Problem Statement 1 (Binary Classification Problem)

To identify if a fellow will ultimately be placed at a company. Since possible outcomes are placed or not placed, hence it is a Binary Classification problem. 

### Identifying Dataset for model building and evaluation

## We split our datasets into two datasets:

### 1). dataset_for_model

Pathrise in interested in those enrolled fellows who are placed within a year. If a fellow is unable to find a job within a year after joining Pathrise, his/her contract is terminated. Column pathrise_status shows different status of candidates.

1). Placed imply that candidate has been placed while enrolled in Pathrise

2). Withdrawn(Failed) imply that candidate did not get any placement even after enrolling in Pathrise for more than one year. 

Therefore, we will consider the placement outcome of these candidates who were with Pathrise over a period of time and we build our model based on this data.

### 2). dataset_for_evaluation

1). MIA candidates are those who missed in action,

2). Withdrawn(Trial) are those candidates who withdrew after a trial period, 

3). Break for those candidates who are on temporary break,

4). Withdrawn for those candidates who withdrew for one reason or other, 

5). Active are still enrolled with Pathrise.

6). Closed Lost who did not accept Pathrise Offer

7). Deferred who accepted offer, but willing to start later

Therfore this data will be used for prediction whether those candidates if continuing/had continued with Pathrise would have got placement or not.


In [293]:
dataset_for_model1=dataset[dataset['pathrise_status']. isin(["Withdrawn (Failed)","Placed"])]
dataset_for_model1.shape

(1038, 11)

In [294]:
dataset_for_evaluation1=dataset[dataset['pathrise_status']. isin(["Active","Break","MIA","Withdrawn (Trial)","Withdrawn","Closed Lost","Deferred"])]
dataset_for_evaluation1.shape

(1506, 11)

### Identifying Dependent and Independent Features for dataset_for_model

In [295]:
X1=dataset_for_model1[["primary_track","employment_status","highest_level_of_education","length_of_job_search","biggest_challenge_in_search","professional_experience","work_authorization_status","number_of_interviews-binned"]]

In [296]:
y1=dataset_for_model1["placed"]

### Splitting dataset into training and test data

In [297]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, train_size=0.8 , random_state=100)

In [298]:
from sklearn.ensemble import RandomForestClassifier 
model1 = RandomForestClassifier()
model1.fit(X1_train, y1_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [299]:
model1.score(X1_test,y1_test)

0.8894230769230769

In [300]:
y1_predicted = model1.predict(X1_test)

In [301]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y1_test, y1_predicted)
cm

array([[  2,  21],
       [  2, 183]])

## Evaluating cross_val_score

In [302]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5)
cross_val_score(RandomForestClassifier(),X1,y1, cv=cv)

array([0.91346154, 0.91346154, 0.95192308, 0.88461538, 0.88461538])

### Accuracy score is nearly 90%. Also, Confusion matrix tells us that model is predicting 183 True positive values out of 208 and ratio of False Positive and False Negative is comparatively negligible. Also, cross_val_score imply that model seems to be a model of good fit.¶

## Deploying Model to predict placement status of fellow

In [303]:
dataset_for_evaluation1=dataset_for_evaluation1[["primary_track","employment_status","highest_level_of_education","length_of_job_search","biggest_challenge_in_search","professional_experience","work_authorization_status","number_of_interviews-binned"]]

In [304]:
predicted_status1=model1.predict(dataset_for_evaluation1)

In [305]:
pd.Series(predicted_status1).value_counts()

1    1490
0      16
dtype: int64

## Conclusion (Problem Statement 1)

### The model predicts that out of 1506 candidates who are either active or have/had continued enrollement with pathrise, 1490 candidates will get/would have been placed ultimately at a company based on their attributes. That means Pathrise has more than 98.9% placement rate.


# Problem Statement 2 

To identify how long until a placement can be expected at a company. 

### The problem at first instance is a Regression problem. But if we classify program_duartion_days into different categories,then it becomes Multi-class Classification Problem.

Our approach will be to deal this problem as a Classification Problem.

### Identifying Dataset for model building and evaluation

#### 1). dataset_for_model
(This includes all those records where pathrise_status of candidates is Placed. That is, we consider data set for placed candidates where program duartion days is already known )

#### 2). dataset_for_evaluation 
(This includes all those records where pathrise_status of candidates is Active or Break or MIA or Withdrawn (Trial) or Deferred or Closed Lost or Withdrawn (in case they had continued with Pathrise)
 
(We will not evaluate program duration days of placement for Withdrawn(Failed) as we already know they didnot get any placement even after one year of continuing with Pathrise)


In [306]:
dataset_for_model2=dataset[dataset['pathrise_status'] =="Placed"]
dataset_for_model2.shape

(956, 11)

In [307]:
dataset_for_evaluation2=dataset[dataset['pathrise_status']. isin(["Active","Break","MIA","Withdrawn (Trial)","Withdrawn","Deferred","Closed Lost"])]
dataset_for_evaluation2.shape

(1506, 11)

### Identifying Dependent and Independent Features for dataset_for_model

In [308]:
X2=dataset_for_model2[["primary_track","employment_status","highest_level_of_education","length_of_job_search","biggest_challenge_in_search","professional_experience","work_authorization_status","number_of_interviews-binned"]]

In [309]:
y2=dataset_for_model2["program_duration_days"]
y2=y2.astype(int)

### Categorize y2 into different Categories

In [310]:
def func2(x):
    if x < 90:
        return "0-3m"
    elif x < 180:
        return "3-6m"
    elif x < 270:
        return "6-9m"
    elif x < 365:
        return "9-12m"
    else:
        return 'more than year'

In [311]:
y2=dataset_for_model2['Classification'] = dataset_for_model2["program_duration_days"].apply(func2)

In [312]:
from sklearn.ensemble import RandomForestClassifier 
model2 = RandomForestClassifier()
model2.fit(X2, y2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [313]:
model2.score(X2,y2)

0.9288702928870293

## Deploying Model to predict program duration days of placed fellow

In [314]:
dataset_for_evaluation2=dataset_for_evaluation2[["primary_track","employment_status","highest_level_of_education","length_of_job_search","biggest_challenge_in_search","professional_experience","work_authorization_status","number_of_interviews-binned"]]

In [315]:
dataset_for_evaluation2.head(2)

,primary_track,employment_status,highest_level_of_education,length_of_job_search,biggest_challenge_in_search,professional_experience,work_authorization_status,number_of_interviews-binned
id,,,,,,,,
1,4,4,0,1,5,1,0,2
2,3,4,5,1,2,0,1,0


In [316]:
predicted_status2=model2.predict(dataset_for_evaluation2)

In [317]:
dataset_for_evaluation2["program_duration_days"]=model2.predict(dataset_for_evaluation2)

In [318]:
dataset_for_evaluation2.head(2)

,primary_track,employment_status,highest_level_of_education,length_of_job_search,biggest_challenge_in_search,professional_experience,work_authorization_status,number_of_interviews-binned,program_duration_days
id,,,,,,,,,
1,4,4,0,1,5,1,0,2,9-12m
2,3,4,5,1,2,0,1,0,9-12m


In [319]:
dataset_for_evaluation2["program_duration_days"].value_counts()

3-6m              523
0-3m              520
6-9m              269
9-12m             181
more than year     13
Name: program_duration_days, dtype: int64

## Conclusion (Problem Statement 2)

### From the above results, we conclude that 70% fellows are placed in first 6 months of their enrollment (35% in first three months, 35% in 3-6 months). A very less percentage(0.008) of candidates will take more than an year to get placed. Hence we can easily conclude that Pathrise is doing well in placement of fellows. And we can highly recommend anyone looking for a placement to join pathrise 

## Recommendation to Pathrise

Pathrise has more than 98.9% placement rate and Fellows at pathrise (whether they are Closed Lost or Deferred or Withdrawn (Trial) or Withdrawn or MIA or Break) are most likely to be placed within one year of their enrollment with 70% of them during first 6 months. And Pathrise is ought to do a good business with the signed contract (Income Share Agreement) within first 12 months. 

Therefore,

1). Fellows at Pathrise  whose status is MIA, Break, Deferred  are encouraged to get back in action by offering some Lucrative offers 

2). Fellows at Pathrise whose status is Withdrawn(Trial), Withdrawn, Closed Lost who are not in any contract with Pathrise can be urged to enroll and accept their offer by relaxing some contract conditions.  